# LIBRARIES & FUNCTIONS

In [2]:
# LIBRARIES #

import numpy as np

from scipy import optimize

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.models import LinearColorMapper, BasicTicker, ColorBar, Plot, CustomJS, ColumnDataSource, Rect, Cross
from bokeh.layouts import row, gridplot, column
from bokeh.models.widgets import Slider, Button
from bokeh.events import ButtonClick

import SimpleITK as sitk 

import pydicom

import json

from Gafchromic import GafchromicFilms as Gaf_RB
from Gafchromic_multichannel import GafchromicFilms as Gaf_multichannel

import time


output_notebook()

Loading BokehJS ...

In [3]:
# FONCTIONS #




# fitting functions for RBGB model:
def funcDose(x, a, b, c, d, e):
    return a*(x[:,0]*x[:,1])+  b*(x[:,0]**c*x[:,1]**d) + e

def funcRBGB(x, a, b, c, d):
    return a*x*x*x +  b*x*x + c*x + d



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleDataArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor]\
                        .reshape((newsizey, subfactor, newsizex, subfactor)).mean(3).mean(1)
    
    return newarray



# subsample data array : #
# @params:
#  array : array to subsample
#  sizex : size in x
#  sizey : size in y
#  subfactor :  subsampling factor
def subSampleRGBArray(array, subfactor):
    newsizex = int(array.shape[1]/subfactor)
    newsizey = int(array.shape[0]/subfactor)

    newarray = array[0:newsizey*subfactor, 0:newsizex*subfactor,:]\
                        .reshape((newsizey, subfactor, newsizex, subfactor, 3)).mean(3).mean(1)
    
    return newarray




# Finds which model from the list of models corresponds best to the rb & gb values
#   
# @params:
#  rb : array of column R/B
#  gb : array of column G/B
#  modelsArr : list of parameters of the models of RBGB curves 
#  dispGraphs : displays graphs if true
def findBestModel(rb, gb, modelsArr, dispGraph=False):
    
    # calculates mean error between model and curve for all models
    errorValues = []
    for i in range(len(modelsArr)):
        mod = modelsArr[i]
        err = np.mean((gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3]))*(gb-funcRBGB(rb, mod[0], mod[1], mod[2], mod[3])))
        errorValues.append(err)
    
    errormin = np.amin(errorValues)
    bestmodel = errorValues.index(errormin)
    
    if dispGraph:
        print("  >>> Best Model:", bestmodel)
        print("      Associated Error:", errormin)
        
        p1 = figure(plot_width=600, plot_height=600, title="RB vs GB", 
                    toolbar_location="above")

        p1.circle(rb, gb, size=3, color='cornflowerblue', alpha=0.3, 
                  legend_label="column = "+str(column)) 

        minrb = np.min(rb)
        maxrb = np.max(rb)
        xRBGB = np.arange(minrb, maxrb, (maxrb-minrb)/50)
        mod = modelsArr[bestmodel]
        p1.line(xRBGB, funcRBGB(xRBGB, mod[0], mod[1], mod[2], mod[3]), 
                line_color="black", line_width=1, line_dash='dotted')

        p1.xaxis.axis_label = "R/B values" 
        p1.yaxis.axis_label = "G/B values"
        p1.legend.location = 'top_left' 
        
        show(p1)
    
    
    return [bestmodel, errormin]




# Converts an image to dose using RBGB models
#   
# @params:
#  arr : image array
#  models : dictionary of RBGB model
def convertToDose(arr, models):
    
    rb = arr[:,:,0]/arr[:,:,2]
    gb = arr[:,:,1]/arr[:,:,2]
    
    doseImg = np.zeros(rb.shape)
    
    rbgbmodels = models["RBGBcurve"]
    for i in range(rb.shape[1]):
        [index, err] = findBestModel(rb[:,i], gb[:,i], rbgbmodels, dispGraph=False)
        
        rbgb = np.zeros((rb.shape[0],2))
        rbgb[:,0] = rb[:,i]
        rbgb[:,1] = gb[:,i]
        mod = models["DoseCurve"][index]
        doseImg[:,i] = funcDose(rbgb, mod[0], mod[1], mod[2], mod[3], mod[4])
    
    return doseImg




# Displays two images and profiles
#   
# @params:
#  img1: image array 1
#  img2: image array 2
#  col: column nb for the profile
#  line: line nb for the profile
def compare2Imgs(img1, img2, col, line, plotwidth=450, title1='dose image 1', title2='dose image 2',
                colorprofile1='firebrick', colorprofile2='darkblue'): 
    
    # Img 1:
    p1 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img1.shape[0]/img1.shape[1]), 
                title=title1, toolbar_location="above")
    p1.image(image=[img1], x=0, y=0, dw=img1.shape[1], dh=img1.shape[0], palette="Plasma256")
    p1.line((col, col), (0, img1.shape[0]), line_alpha=0.7, line_color="white")
    p1.line((0, img1.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Img 2
    p2 = figure(plot_width=plotwidth, plot_height=int(plotwidth*img2.shape[0]/img2.shape[1]), 
               title=title2, toolbar_location="above")
    p2.image(image=[img2], x=0, y=0, dw=img2.shape[1], dh=img2.shape[0], palette="Plasma256")
    p2.line((col, col), (0, img2.shape[0]), line_alpha=0.7, line_color="white")
    p2.line((0, img2.shape[1]), (line, line), line_alpha=0.7, line_color="white")


    # Horizontal profile:
    maxx = np.amax(img1[line,:])
    if np.amax(img2[line,:])>maxx: maxx = np.amax(img2[line,:])
        
    p3 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="x profile", 
                toolbar_location="above", y_range=(0, int(1.05*maxx)))
    x3 = np.arange(0, len(img1[line,:]), 1)
    x3b = np.arange(0, len(img2[line,:]), 1)
    p3.line(x3, img1[line,:], line_width=2, line_color=colorprofile1, legend_label=title1)
    p3.line(x3b, img2[line,:], line_width=2, line_color=colorprofile2, legend_label=title2)


    # Vertical profile:
    p4 = figure(plot_width=plotwidth, plot_height=int(plotwidth*2/3), title="y profile", toolbar_location="above")
    x4 = np.arange(0, len(img1[:,col]), 1)
    x4b = np.arange(0, len(img2[:,col]), 1)
    p4.line(x4, img1[:, col], line_width=3, line_color=colorprofile1, legend_label=title1)
    p4.line(x4b, img2[:, col], line_width=3, line_color=colorprofile2, legend_label=title2)

    grid = gridplot([[p1, p2], [p3, p4]])


    show(grid)

    

    
# Displays two images and profiles
#   
# @params:
#  array: image array on which to perform streak correction.
def applyStreakCorrection(array):
        
        # First pixels sum and normalization:
        firstPixAvg_r = np.sum(array[0:10,:,0] ,axis=0)
        firstPixAvg_g = np.sum(array[0:10,:,1] ,axis=0)
        firstPixAvg_b = np.sum(array[0:10,:,2],axis=0)

        firstPixAvg_r = firstPixAvg_r / np.mean(firstPixAvg_r)
        firstPixAvg_g = firstPixAvg_g / np.mean(firstPixAvg_g)
        firstPixAvg_b = firstPixAvg_b / np.mean(firstPixAvg_b)


        # Last pixels sum and normalization:
        lastPixAvg_r = np.sum(array[-9:,:,0] ,axis=0)
        lastPixAvg_g = np.sum(array[-9:,:,1] ,axis=0)
        lastPixAvg_b = np.sum(array[-9:,:,2],axis=0)

        lastPixAvg_r = lastPixAvg_r / np.mean(lastPixAvg_r)
        lastPixAvg_g = lastPixAvg_g / np.mean(lastPixAvg_g)
        lastPixAvg_b = lastPixAvg_b / np.mean(lastPixAvg_b)


        # Streak correction image:
        #  This correction must be done on an uncroped image
        streakCorrImage = np.zeros(array.shape)
        x = [0, array.shape[1]-1]

        for i in range(array.shape[1]):
            allpix_y = range(array.shape[0])
            y0 = [firstPixAvg_r[i], lastPixAvg_r[i]]
            streakCorrImage[:,i,0] = 1 / np.interp(allpix_y, x, y0)
    
            y1 = [firstPixAvg_g[i], lastPixAvg_g[i]]
            streakCorrImage[:,i,1] = 1 / np.interp(allpix_y, x, y1)

            y2 = [firstPixAvg_b[i], lastPixAvg_b[i]]
            streakCorrImage[:,i,2] = 1 / np.interp(allpix_y, x, y2)
    
    
        # Apply streak correction to the image:
        return array * streakCorrImage

    

# Conversion to dose:

In [17]:
# VARIABLES :

m_path = 'testRBGB/'
m_nbOfFiles = 1
m_firstNb = 1
m_GafFilesName = "testRBGB_00"
m_extension = ".tif"

m_subFactor = 10

m_RBGBFile = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center.txt"
# m_RBGBFile2 = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_mean.txt"
m_RBGBFile2 = "C:/Users/00004436/Documents/GitHub/Gafchromic_films/testRBGB/calibRBGB_center_overlappingLines.txt"

m_splineFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/bSpline_data.txt'
m_multiLinearFile = 'G:/Commun/PHYSICIENS/Erwann/EBT3/13 - etalonnage lot 02282001/scan 24h/multiLinearRegressionCoefficients_02282001.txt'




In [5]:
# READS THE GAF FILE and converts it using RBGB model:


# Reads the first image:
img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb)+m_extension)

sizex = img.GetWidth()
sizey = img.GetHeight()
imgOrigin = img.GetOrigin()
imgSpacing = img.GetSpacing()


# Reads all images and does the median image:
size = (sitk.GetArrayFromImage(img).shape[0], 
        sitk.GetArrayFromImage(img).shape[1], 
        sitk.GetArrayFromImage(img).shape[2], 
        m_nbOfFiles)
imgs = np.zeros(size)

for i in range(m_nbOfFiles):
    img = sitk.ReadImage(m_path+m_GafFilesName+str(m_firstNb+i)+m_extension)
    imgs[:,:,:,i] = sitk.GetArrayFromImage(img)
    
array = np.median(imgs, axis=3)

array = subSampleRGBArray(array, m_subFactor)

# displays the image to select the area to convert:
# p1 = figure(plot_width=600, plot_height=int(600*array.shape[0]/array.shape[1]), 
#            title='Image to convert', toolbar_location="above")
# p1.image(image=[array[:,:,0]], x=0, y=0, dw=array.shape[1], dh=array.shape[0], palette="Plasma256")

# show(p1)


In [11]:
# Convert an image to dose using the models:



# Image to convert to dose: [y1:y2, x1:x2, :]
arrayToConvert = array[40:300, 50:260, :]

# Reading RBGB models:
with open(m_RBGBFile, 'r') as file:
    gafModelsDict = json.load(file)

with open(m_RBGBFile2, 'r') as file:
    gafModelsDict2 = json.load(file)

    
# conversion using the models:
doseimg_RBGB = convertToDose(arrayToConvert, gafModelsDict)

doseimg_RBGB2 = convertToDose(arrayToConvert, gafModelsDict2)

print("  > Image converted to dose using RBGB model.")


# conversion using classic R/B:
try:
    g = Gaf_RB(m_path+m_GafFilesName, m_firstNb, m_nbOfFiles)
    g.subSampleDataArray(10)
    a = g.cropImg(50,260,40,300)
    doseimg_basicRB = g.convertToDose_cubicSplineFit(m_splineFile, 1000)
    size = g.getSize()
except ValueError as err:
    print('Erreur: ' + err)

print("  > Image converted to dose using basic RB model.")



  > Image converted to dose using RBGB model.
  > Image converted to dose using basic RB model.


In [16]:
# shows dose images:

compare2Imgs(doseimg_RBGB2, doseimg_basicRB, 50, 180, title1="calibRBGB_center_overlappingLines", title2="basic R/B calibration")

In [ ]:
# A FAIRE: COMPARAISON AVEC ETALONNAGE MULTI LINEAIRE, AJOUTER IMAGES AU MODELE POUR PLUS DE LIGNES DANS MODELE
#   VOIR AVEC AUTRE FILMS POUR SAVOIR SI CA FONCTIONNE ENCORE, VOIR POUR AUTRE RESOLUTION ET BRUIT...